In [145]:
from base64 import b64encode, b64decode
from Crypto.Util import number
from random import randint
import sys


In [347]:
class RSASender:
    def __init__( self , public_key ):
        self.public_key = public_key

    def encrypt(self,message):
        enc = b''
        len_e = self.public_key[0:4]
        len_e = int.from_bytes(len_e, 'big')

        e = int.from_bytes(self.public_key[4:4+len_e],'big')

        len_N = self.public_key[len_e+4:len_e+4+4]

        len_N =int.from_bytes(len_N, 'big') 

        N = int.from_bytes(self.public_key[len_e+4+4:len_e+4+4+len_N],'big')

        msg_arr = bytearray(message, 'utf-8')

        n_bits = len_N*8
        
        n = int((n_bits-1)/8)

        message_len = len(msg_arr)
        
        for i in range(0,message_len,n):
            a = int.from_bytes(msg_arr[i:i + n], 'big')
            c = pow(a,e,N)
            c_bytes = c.to_bytes(n+1,'big')
            enc += c_bytes
            
        return(enc)
        



In [356]:
class RSAReciever:
    def __init__( self , bit_len ):
        self.bit_len = bit_len
        self.d=0
        self.N = 0
        self.N_bytes = 0
        self.e = 0

    def _exgcd(self, a, b):
        """Extended Euclidean Algorithm that can give back all gcd, s, t 
        such that they can make Bézout's identity: gcd(a,b) = a*s + b*t
        Return: (gcd, s, t) as tuple"""
        old_s, s = 1, 0
        old_t, t = 0, 1
        while b:
            q = a // b
            s, old_s = old_s - q * s, s
            t, old_t = old_t - q * t, t
            a, b = b, a % b
        return a, old_s, old_t
        
    def _gcd(self,a, b):
        '''Computes the Great Common Divisor using the Euclid's algorithm'''
        while b:
            a, b = b, a % b
        return a

    def _lcm(self,a, b):
        """Computes the Lowest Common Multiple using the GCD method."""
        return a // self._gcd(a, b) * b

    def _invmod(self,e, m):
        """Find out the modular multiplicative inverse x of the input integer
        e with respect to the modulus m. Return the minimum positive x"""
        g, x,y= self._exgcd(e, m)
        assert g == 1

        # Now we have e*x + m*y = g = 1, so e*x ≡ 1 (mod m).
        # The modular multiplicative inverse of e is x.
        if x < 0:
            x += m
        return x

    def get_public_key(self):
        key = b''
        t = 0
        
        mcd = self._gcd(self.d, t)
        P = number.getPrime(self.bit_len//2)
        Q = number.getPrime(self.bit_len//2)
        N = P*Q
        self.N = N
        t = (P-1)*(Q-1)
        while mcd != 1:   
            self.d = randint(2,N-1)
            mcd = self._gcd(self.d, t)

        e = self._invmod(self.d, t)
        self.e = e

        len_e = (e.bit_length()+7)//8
        d_bytes = e.to_bytes(len_e,'big')

        len_N = (N.bit_length()+7)//8
        self.N_bytes = len_N
        N_bytes = N.to_bytes(len_N,'big')

        len_N = len_N.to_bytes(4,'big')
        len_e = len_e.to_bytes(4,'big')
        key += len_e
        key += d_bytes
        key += len_N
        key += N_bytes



        return key
    

    def decrypt(self,cyphertext):

        decrypt_msg = ''
        n_bits = self.N_bytes*8
        n = int((n_bits-1)/8)+1

        for i in range(0,len(cyphertext),n):
            aux = cyphertext[i:i+n]
            m = int.from_bytes(aux,'big')
            
            f = pow(m,self.d,self.N)

            decrypt_msg += f.to_bytes(n-1,'big').decode('utf-8').strip('\x00')
            
        return decrypt_msg